In [1]:
!pip install openpyxl

In [2]:
!pip install --upgrade jupyter ipywidgets

  Using cached importlib_metadata-7.0.1-py3-none-any.whl (23 kB)
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 3.10.1
    Uninstalling importlib-metadata-3.10.1:
      Successfully uninstalled importlib-metadata-3.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
konoha 4.6.5 requires importlib-metadata<4.0.0,>=3.7.0, but you have importlib-metadata 7.0.1 which is incompatible.


In [275]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [276]:
xlsx_file ='/u1/mdr614/ATUC_Final/Prepared_models/Detection Models/Review comment Toxicity detection models/Dataset/My Data/for_Train_and_validation_data.xlsx' 
df = pd.read_excel(xlsx_file)
df.head()

,message,is_toxic
0,This and below assignments also should be removed,0
1,this should be flavor_id = self.flavor_id,0
2,I am confused.\n \n This is the tar process we...,0
3,"Yuck. Use %.70s, which will do this more grace...",1
4,Use mock_thread_instance = mock.MagicMock(isAl...,0


In [277]:
df.is_toxic.value_counts()

0    16403
1     5715
Name: is_toxic, dtype: int64

In [278]:
possible_labels = df.is_toxic.unique()
possible_labels

array([0, 1])

In [279]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [280]:
label_dict

{0: 0, 1: 1}

In [281]:
df['label'] = df.is_toxic.replace(label_dict)

In [282]:
df.head()

,message,is_toxic,label
0,This and below assignments also should be removed,0,0
1,this should be flavor_id = self.flavor_id,0,0
2,I am confused.\n \n This is the tar process we...,0,0
3,"Yuck. Use %.70s, which will do this more grace...",1,1
4,Use mock_thread_instance = mock.MagicMock(isAl...,0,0


In [283]:
from sklearn.model_selection import train_test_split

In [284]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.15, 
                                                  random_state=17, 
                                                  stratify=df.label.values)

In [285]:
X_train

array([ 9294,  9118, 11070, ..., 11111,  2284, 18581])

In [286]:
df['data_type'] = ['not_set']*df.shape[0]

In [287]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [288]:
df

,message,is_toxic,label,data_type
0,This and below assignments also should be removed,0,0,train
1,this should be flavor_id = self.flavor_id,0,0,val
2,I am confused.\n \n This is the tar process we...,0,0,val
3,"Yuck. Use %.70s, which will do this more grace...",1,1,train
4,Use mock_thread_instance = mock.MagicMock(isAl...,0,0,train
...,...,...,...,...
22113,"Unpredictable behavior, courtesy of your code.",1,1,train
22114,Coding with a dash of unpredictability.,1,1,train
22115,Your commits: keeping us all guessing!,1,1,train
22116,"The app's new motto: ""Expect Anything!""",1,1,train


In [289]:
df.groupby(['is_toxic', 'label', 'data_type']).count()

message
is_toxic label data_type         
0        0     train        13942
               val           2461
1        1     train         4858
               val            857

In [290]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [291]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [292]:
df[df.data_type=='train'].message.values

array(['This and below assignments also should be removed',
       'Yuck. Use %.70s, which will do this more gracefully',
       'Use mock_thread_instance = mock.MagicMock(isAlive=mock.MagicMock(return_value=False))\n \n remove L69',
       ..., 'Your commits: keeping us all guessing!',
       'The app\'s new motto: "Expect Anything!"',
       'Your coding style: mystery or mastery?'], dtype=object)

In [293]:
df.iloc[X_val[0]]

message      Hang on.. logic might not be good here.
is_toxic                                           0
label                                              0
data_type                                        val
Name: 2922, dtype: object

In [294]:
df.iloc[y_val[0]]

message      This and below assignments also should be removed
is_toxic                                                     0
label                                                        0
data_type                                                train
Name: 0, dtype: object

In [295]:
X_val[0]

2922

In [296]:
# from transformers import AutoTokenizer
# import torch
# import pandas as pd

# # Replace 'model_name' with the name of the pre-trained model you're using
# model_name = 'bert-base-uncased'
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# # Assuming you have a DataFrame named 'df'
# for index, row in df.iterrows():
#     data_type = row['data_type']
#     message = row['message']
#     label = row['label']

#     encoded_data = tokenizer.encode_plus(
#         text=message,
#         add_special_tokens=True,
#         return_attention_mask=True,
#         pad_to_max_length=True,
#         max_length=256,
#         return_tensors='pt'
#     )

#     input_ids = encoded_data['input_ids']
#     attention_mask = encoded_data['attention_mask']
    
#     #print(f"Processing row {index + 1} - Data Type: {data_type}, Message: {message}, Label: {label}")
    
#     # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'
 

In [297]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=1024, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].message.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=1024, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [298]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [299]:
len(dataset_train)

18800

In [300]:
len(dataset_val)

3318

In [301]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [302]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [303]:
batch_size = 32

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [304]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [305]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [306]:
epochs = 8

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [307]:
import numpy as np
from sklearn.metrics import f1_score

In [308]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [309]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [310]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [311]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [312]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [313]:
# for epoch in tqdm(range(1, epochs+1)):
    
#     model.train()
    
#     loss_train_total = 0

#     progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
#     for batch in progress_bar:

#         model.zero_grad()
        
#         batch = tuple(b.to(device) for b in batch)
        
#         inputs = {'input_ids':      batch[0],
#                   'attention_mask': batch[1],
#                   'labels':         batch[2],
#                  }       

#         outputs = model(**inputs)
        
#         loss = outputs[0]
#         loss_train_total += loss.item()
#         loss.backward()

#         torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

#         optimizer.step()
#         scheduler.step()
        
#         progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
#     torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
#     tqdm.write(f'\nEpoch {epoch}')
    
#     loss_train_avg = loss_train_total/len(dataloader_train)             
#     tqdm.write(f'Training loss: {loss_train_avg}')
    
#     val_loss, predictions, true_vals = evaluate(dataloader_validation)
#     val_f1 = f1_score_func(predictions, true_vals)
#     tqdm.write(f'Validation loss: {val_loss}')
#     tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [314]:
# val_loss, predictions, true_vals = evaluate(dataloader_validation)

In [315]:
# preds_flat = np.argmax(predictions, axis=1).flatten()

In [316]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [318]:
model.load_state_dict(torch.load('/u1/mdr614/ATUC_Final/Prepared_models/Detection Models/Review comment Toxicity detection models/MyModel_BERT.model', map_location=torch.device('cpu')))

<All keys matched successfully>

In [319]:
# _, predictions, true_vals = evaluate(dataloader_validation)

In [320]:
# accuracy_per_class(predictions, true_vals)

In [321]:
# len(predictions), len(true_vals)

In [322]:
# from sklearn import metrics

In [323]:
# print("Accuracy:",metrics.accuracy_score(preds_flat, true_vals))
# print("Precision:",metrics.precision_score(preds_flat, true_vals))
# print("Recall:",metrics.recall_score(preds_flat, true_vals))
# print("f1 score", metrics.f1_score(preds_flat, true_vals, average='weighted'))

### Testing

In [55]:
# #please create a new column "is_toxic" and fill the first row with 1 and rest with 0. This is just a dummy column. 
#You can delete it after the operation

In [399]:
xlsx_file ='/u1/mdr614/ATUC_Final/Conversion_uncivil_to_civil_random_2000/Incivility_conversion_results/civilAlternatives_by_Paragedi_random2000.csv' 
#df_test = pd.read_excel(xlsx_file)
df_test = pd.read_csv(xlsx_file)
df_test.head()

,uncivil,civil,is_toxic
0,eapByteBuffer[pos=0 lim=0 cap=0]]] output: Byt...,eapByteBuffer[pos=0 lim=0 cap=0]] output: Byte...,1
1,"It looks like this will result in a fork, whic...",it looks like this will lead to a fork in the ...,0
2,between phones as a loss. The inconsistency i...,The lack of consistency in my own ecosystem th...,0
3,"|| checkLineNo(field, objCalledOn))) { > ``` ...",if (field!= null&& (frame.isClassOrEnumOrEnumc...,0
4,No one from this list ever responds,no one on the list ever replied.,0


In [400]:
possible_labels = df_test.is_toxic.unique()
possible_labels

array([1, 0])

In [401]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [402]:
df_test['label'] = df_test.is_toxic.replace(label_dict)

In [403]:
from transformers import AutoTokenizer
import torch
import pandas as pd

# Replace 'model_name' with the name of the pre-trained model you're using
model_name = 'bert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Assuming you have a DataFrame named 'df'
for index, row in df_test.iterrows():

    message = row['civil']


    encoded_data = tokenizer.encode_plus(
        text=message,
        add_special_tokens=True,
        return_attention_mask=True,
        pad_to_max_length=True,
        max_length=256,
        return_tensors='pt'
    )

    input_ids = encoded_data['input_ids']
    attention_mask = encoded_data['attention_mask']
    
    print(f"Processing row {index + 1} -  Message: {message}")
    
    # Here you can proceed with storing or processing 'input_ids', 'attention_mask', and 'label'

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/mdr614/anaconda3/envs/toxic_gru/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2619: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Processing row 1 -  Message: eapByteBuffer[pos=0 lim=0 cap=0]] output: ByteBufferWrapper (884587465)
Processing row 2 -  Message: it looks like this will lead to a fork in the road, which is a real shame, but that's what happens if you throw your community off the boat.
Processing row 3 -  Message: The lack of consistency in my own ecosystem though.
Processing row 4 -  Message: if (field!= null&& (frame.isClassOrEnumOrEnumconstDef() || checkLineNo(field, OBjCalledOn)) > object Concatenations would not cause NullPointerException even though "object" was mentioned
Processing row 5 -  Message: no one on the list ever replied.
Processing row 6 -  Message: I PushTextSendJob: [B79876d5-f4c8-44c7-84d6-9e3f956b74dc] Sent message: 36998 (Time since Submission: 720847 ms, Lifespan:
Processing row 7 -  Message: Of course, this is not a complaint, just a suggestion, please do not take it that way.
Processing row 8 -  Message: * If the Rstudio team is indeed set to use electron, rather than qtweben

In [404]:

encoded_test_val = tokenizer.batch_encode_plus(
    df_test.civil.values.tolist(), 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=256, 
    return_tensors='pt'
)

In [405]:
input_ids_test = encoded_test_val['input_ids']
attention_masks_test = encoded_test_val['attention_mask']
labels_test = torch.tensor(df_test.label.values)

In [406]:
dataset_test = TensorDataset(input_ids_test, attention_masks_test,labels_test)
batch_size = 32
dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [407]:
def test_model(dataloader_test):

    model.eval()
    
    predictions = []
    
    for batch in dataloader_test:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                'attention_mask': batch[1],
                'labels':         batch[2],
                }  
        with torch.no_grad():        
            outputs = model(**inputs)
            
        logits = outputs[1]

        logits = logits.detach().cpu().numpy()
        predictions.append(logits)
    
    predictions = np.concatenate(predictions, axis=0)
    return predictions

In [408]:
pred_test = test_model(dataloader_test) 
preds_flat_test = np.argmax(pred_test, axis=1).flatten()
print(preds_flat_test)

[0 1 0 ... 0 1 0]


In [409]:
import pandas as pd
pred_data = pd.DataFrame(preds_flat_test, columns = ["prediction"])
pd_cont = pd.concat([df_test["civil"], pred_data], axis = 1)
pd_cont.to_csv("civilAlternatives_by_Paragedi_random2000_incivilityDetection_byMyModel.csv")